In [1]:
import h5py
import time
import numpy as np
import torch as T
from Pyccat_v1 import CCAT_holo

DEVICE0=T.device('cuda:0')
measuredbeams='input_beams/GRASP_296GHz_101_101.txt'

In [10]:
'''test x'''
model=CCAT_holo('CCAT_MODEL','output0')
#model.build_holo()
model.mk_FF_4maps(Device=DEVICE0,Memory_reduc=True)

In [11]:
S0=np.genfromtxt('input_beams/input_error_panel.txt')
S0=np.zeros(5*146)
paraA=T.tensor([1.0,0,0,0,0,0]*4,dtype=T.float64).to(DEVICE0)
paraP=T.tensor(np.zeros(5*4),dtype=T.float64).to(DEVICE0)
start=time.time()
x1=model.FF(T.tensor(S0,dtype=T.float64).to(DEVICE0),paraA,paraP)
print(time.time()-start) 

0.019348859786987305


In [13]:
start=time.time()
for n in range(100):
    x1=model.FF(T.tensor(S0,dtype=T.float64).to(DEVICE0),paraA,paraP)
print(time.time()-start) 

7.267016410827637


In [ ]:
model.mk_FF_4maps(Device=DEVICE0)

In [ ]:
S0=np.genfromtxt('input_beams/input_error_panel.txt')
S0=np.zeros(5*146)
paraA=T.tensor([1.0,0,0,0,0,0]*4,dtype=T.float64).to(DEVICE0)
paraP=T.tensor(np.zeros(5*4),dtype=T.float64).to(DEVICE0)
start=time.time()
x2=model.FF(T.tensor(S0,dtype=T.float64).to(DEVICE0),paraA,paraP)
print(time.time()-start) 

In [ ]:
x1=x1.data.cpu()
x2=x2.data.cpu()

In [ ]:
print(T.tensor(np.flip(x2[4,:].numpy().reshape(101,-1),axis=1).ravel())-x1[4,:])
print(T.tensor(np.flip(x2[5,:].numpy().reshape(101,-1),axis=1).ravel())-x1[5,:])
print(T.tensor(np.flip(x2[6,:].numpy().reshape(101,-1),axis=1).ravel())-x1[6,:])
print(T.tensor(np.flip(x2[7,:].numpy().reshape(101,-1),axis=1).ravel())-x1[7,:])

In [ ]:
r1=(x1[0:4,:]-x2[0:4,:])

In [ ]:
for n in range(2):
    print(x1[2*n,:].numpy()-x2[2*n,:].numpy())
    print(x1[2*n+1,:].numpy()-x2[2*n+1,:].numpy())
for n in range(2,4):
    print(T.flip(x1[2*n,:].reshape(101,-1),[1]).ravel()-x2[2*n,:])
    print(T.flip(x1[2*n+1,:].reshape(101,-1),[1]).ravel()-x2[2*n+1,:])

In [ ]:
T.flip(x1,[1])

In [ ]:
for n in range(2):
    print(x1[2*n,:].numpy()-x2[2*n,:].numpy())
    print(x1[2*n+1,:].numpy()-x2[2*n+1,:].numpy())
for n in range(2,4):
    print(np.flip(x1[2*n,:].numpy().reshape(101,-1),axis=1).ravel()-x2[2*n,:].numpy())
    print(np.flip(x1[2*n+1,:].numpy().reshape(101,-1),axis=1).ravel()-x2[2*n+1,:].numpy())

In [ ]:
Meas=T.tensor(np.genfromtxt(measuredbeams,delimiter=','),dtype=T.float64)
model.fit_LP(Meas,Device=DEVICE0,Map_N=4)
np.savetxt('OLD/output/inference/test/results_LP'+'.txt',model.result_LP.x)

In [ ]:
[1e-2,1e-1,1e-2,1e-2,1e-2,5]

In [ ]:
Meas=T.tensor(np.genfromtxt(measuredbeams,delimiter=','),dtype=T.float64)
model.fit_surface(Meas,Device=DEVICE0,constraint=[1,1,1,1,1,1,10],Map_N=4,Init_LP=model.result_LP.x)
np.savetxt('OLD/output/inference/test/results_test_LP'+'.txt',model.result.x)

In [ ]:
Meas=T.tensor(np.genfromtxt(measuredbeams,delimiter=','),dtype=T.float64)
model.fit_surface(Meas,Device=DEVICE0,constraint=[1e-2,1e-1,1e-2,1e-2,1e-2,1],Map_N=4,Init_LP=np.append(np.array([1,0,0,0,0,0]*4),np.array([0,0,0,0,0]*4)))
np.savetxt('OLD/output/inference/test/results_test'+'.txt',model.result.x)

In [ ]:
model.result.x

In [ ]:
(model.result.x[0:-44]-S0).max()*1000

In [ ]:
T.device('cuda')